# Testeando el padData y el createIndexation

## Paquetes necesarios

In [1]:
import transformData3 as td
import matplotlib.pyplot as plt
import numpy as np
import pycblosc2 as cb2
import time as t

## Definición del conjunto de datos

### Datos sin transformación

In [2]:
SHAPE = [8, 8, 8]

SIZE = np.prod(SHAPE)

src = np.arange(SIZE, dtype=np.int32).reshape(SHAPE)

ITEMSIZE = src.itemsize

### Datos transformados

In [10]:
PART_SHAPE = [4, 4, 4]

src_part, indexation = td.tData(src, PART_SHAPE, inverse=False)

TSHAPE = src_part.shape

In [11]:
KB = 1024
MB = 1024 * KB
GB = 1024 * MB

BLOSC_MAX_FILTERS = 5
BLOSC_BLOCKSIZE = 16 * KB
BLOSC_TYPESIZE = 4
BLOSC_CODE = 5

cparams = cb2.blosc2_create_cparams(compcode=BLOSC_CODE, clevel=8, use_dict=0, typesize=BLOSC_TYPESIZE,
                                        nthreads=4, blocksize= BLOSC_BLOCKSIZE, schunk=None,
                                        filters=[0, 0, 0, 0, 1],
                                        filters_meta=[0, 0, 0, 0, 0])

dparams = cb2.blosc2_create_dparams(nthreads=4, schunk=None)

In [12]:
start = t.perf_counter()

schunk = td.compress(cparams, dparams, src)

end = t.perf_counter()

torig_com = end - start

In [19]:
start = t.perf_counter()

res = td.decompress(schunk, ITEMSIZE, SHAPE, b=3)

end = t.perf_counter()

torig_dec = end - start

In [20]:
start = t.perf_counter()

schunk_t = td.compress_trans(cparams, dparams, src_part, TSHAPE, PART_SHAPE)

end = t.perf_counter()

ttran_com = end - start

In [21]:
start = t.perf_counter()

res2 = td.decompress_trans(schunk_t, indexation, ITEMSIZE, SHAPE, TSHAPE, PART_SHAPE,  b=3)

end = t.perf_counter()

ttran_dec = end - start

In [22]:
res2

[1, 1, 1, 1, 1, 8, 8, 8]

In [23]:
res

array([[[[[[[80, 81, 82, 83, 88, 89, 90, 91],
            [ 0,  0,  0,  0,  0,  0,  0,  0],
            [ 0,  0,  0,  0,  0,  0,  0,  0],
            [ 0,  0,  0,  0,  0,  0,  0,  0],
            [ 0,  0,  0,  0,  0,  0,  0,  0],
            [ 0,  0,  0,  0,  0,  0,  0,  0],
            [ 0,  0,  0,  0,  0,  0,  0,  0],
            [ 0,  0,  0,  0,  0,  0,  0,  0]]]]]]], dtype=int32)